In [1]:
# from pyspark.sql import SparkSession, Row
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
# spark = SparkSession.builder.getOrCreate()

In [2]:
class StringConverter(dict):
    def __contains__(self, item):
        return True

    def __getitem__(self, item):
        return str

    def get(self, default=None):
        return str

In [ ]:
bank_accounts = pd.read_csv("fraud_shopee\\bank_accounts.csv", converters=StringConverter())
credit_card = pd.read_csv("fraud_shopee\\credit_cards.csv", converters=StringConverter())
devices = pd.read_csv("fraud_shopee\\devices.csv", converters=StringConverter())
orders = pd.read_csv("fraud_shopee\\orders.csv", converters=StringConverter())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Giao dịch giả mạo được định nghĩa là các giao dịch trong đó người mua và người bán là một cá nhân(Trong thực tế). Dữ liệu được cung cấp là dữ liệu mẫu cho các giao dịch và thông tin chi tiết cuả người đung sẽ được cung cấp.

Nhiệm vụ: Tìm các đơn hàng giả mạo của người mua và người bán được liên kết trực tiếp bằng bất kỳ liên kết nào sau đây: Thiết bị, Thẻ tín dụng và Tài khoản ngân hàng.

+ Liên kết trực tiếp: người mua và người bán chia sẻ cùng một chi tiết

+ Liên kết gián tiếp: ngươi mua và người bán không được liên kết trực tiếp, nhưng người dùng có cùng chi tiết với họ chia sẻ với nhau. Ví dụ: người mua - người dùng A - người dùng B - người dùng C - ... - người dùng Z - người bán

Ví dụ: đơn hàng 28, người mua 36 và người bán 52 nếu người mua và người bán cùng chung thiết bị, hai tài khoản nhiễu --> Gian lận

Ví dụ: đơn hàng 30, người mua 64 và người bán 72

64 liên kết với 80 bằng 1 thiết bị

80 liên kết với 72 bằng 1 thiết bị

--> 64 và 72 là cùng chung một thiết bị nhưng nhiều tài khoản nhiễu

### Bank_accounts

In [ ]:
bank_accounts

,userid,bank_account
0,21829134,923302000003892
1,95910542,011002023212822
2,96941876,0189303223
3,23452396,2280003199803
4,12647942,032002028484803
...,...,...
350836,23568930,084902023320802
350837,17207934,023102020991822
350838,23862272,0090343142
350839,23983072,3330993800


In [ ]:
duplicates = bank_accounts.duplicated(subset=['bank_account'], keep=False)
bank_accounts[duplicates==False]

,userid,bank_account
0,21829134,923302000003892
1,95910542,011002023212822
2,96941876,0189303223
3,23452396,2280003199803
4,12647942,032002028484803
...,...,...
350836,23568930,084902023320802
350837,17207934,023102020991822
350838,23862272,0090343142
350839,23983072,3330993800


In [ ]:
bank_accounts.groupby('bank_account').count().count()

userid    328543
dtype: int64

In [ ]:
bank_accounts[duplicates==True].groupby('bank_account').count()

,userid
bank_account,
0000000000,3
0000302320082342,2
000102023399822,2
000102031339809,2
0001144911200,2
...,...
999302022822801,3
9998008214,2
999802000402802,2


In [ ]:
bank_accounts[duplicates==False].groupby('bank_account').count()

,userid
bank_account,
0,1
0000000000220,1
0000000010101,1
0000000240224,1
0000002202199,1
...,...
99999991,1
9999999200,1
9999999931,1


In [ ]:
bank_accounts[bank_accounts['userid'] == '100004114']

,userid,bank_account
328135,100004114,0802288333
343045,100004114,0939023434
343432,100004114,9292890282


In [ ]:
bank_accounts[bank_accounts['bank_account'] == '999302022822801']

,userid,bank_account
98438,120863728,999302022822801
195692,54963664,999302022822801
292832,202580470,999302022822801


In [ ]:
user_bank_account = bank_accounts.duplicated(subset='userid', keep = False)
bank_accounts.groupby('userid').count()

,bank_account
userid,
10000108,1
10000114,1
100001610,1
100001712,3
100001882,1
...,...
99994206,1
99995186,1
99997576,1


In [ ]:
bank_accounts[user_bank_account == False].groupby('userid').count()

,bank_account
userid,
10000108,1
10000114,1
100001610,1
100001882,1
100003150,1
...,...
99994206,1
99995186,1
99997576,1


In [ ]:
bank_accounts[user_bank_account == True].groupby('userid').count()

,bank_account
userid,
100001712,2
100004114,2
100005624,1
100007056,3
100007258,2
...,...
9998422,2
9998470,1
9998480,1


In [ ]:
bank_accounts[bank_accounts.duplicated()]

,userid,bank_account
17103,165366362,9340289132
18476,53784054,3430499993
18869,8441264,0320142333
34053,99141812,3032984244
37755,108882620,2940831490
...,...,...
347338,31010794,0299203022
347398,168409446,4299821982
347510,105119648,2998238092
349331,163114748,0230082911


In [ ]:
bank_accounts[bank_accounts.duplicated() == False]

,userid,bank_account
0,21829134,923302000003892
1,95910542,011002023212822
2,96941876,0189303223
3,23452396,2280003199803
4,12647942,032002028484803
...,...,...
350836,23568930,084902023320802
350837,17207934,023102020991822
350838,23862272,0090343142
350839,23983072,3330993800


In [ ]:
bank_accounts[bank_accounts['userid'] == '65921462']

,userid,bank_account
4007,65921462,0332420982
82860,65921462,0332420982


### Credit_card

In [ ]:
credit_card

,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018
...,...,...
38703,25879226,839199xxxxxx3838|12-2022
38704,9331492,398138xxxxxx2208|08-2019
38705,2890486,839330xxxxxx9180|08-2022
38706,107429252,899933xxxxxx2930|06-2021


In [ ]:
credit_card.groupby('userid').count()

,credit_card
userid,
10000114,1
100007258,1
100038898,1
100042654,1
10004324,2
...,...
99964354,1
9997318,1
9998480,2


In [ ]:
credit_card[credit_card.duplicated(subset='userid', keep = False)].groupby('userid').count()

,credit_card
userid,
10004324,2
10004484,2
10004798,2
10010266,3
10011214,3
...,...
9985462,3
9989074,2
9993016,2


In [ ]:
credit_card.groupby('credit_card').count()

,userid
credit_card,
300243xxxxxx2422|08-2020,1
300243xxxxxx2918|08-2021,1
300243xxxxxx9830|12-2020,1
301348xxxxxx0010|02-2021,1
301348xxxxxx0033|10-2017,1
...,...
899949xxxxxx9332|12-2017,1
899949xxxxxx9430|10-2017,1
899949xxxxxx9430|10-2022,1


In [ ]:
credit_card[credit_card.duplicated(subset='credit_card', keep = False)].groupby('credit_card').count()

,userid
credit_card,
301348xxxxxx1098|01-2020,3
301348xxxxxx2139|11-2022,2
301348xxxxxx2298|12-2017,3
301348xxxxxx3043|06-2020,2
301348xxxxxx3292|10-2022,2
...,...
899293xxxxxx3191|11-2018,2
899933xxxxxx8433|11-2021,2
899949xxxxxx0092|07-2020,2


In [ ]:
credit_card[credit_card['credit_card'] == '301348xxxxxx1098|01-2020']

,userid,credit_card
8354,24704966,301348xxxxxx1098|01-2020
20016,14202676,301348xxxxxx1098|01-2020
36470,46012496,301348xxxxxx1098|01-2020


In [ ]:
credit_card[credit_card.duplicated()]

,userid,credit_card


### Devices

In [ ]:
devices

,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=
...,...,...
1430306,7061456,ZSTXD6wfHN/1qPVqMrIbwHYX5ELWlUPO3NFXCHK50Mc=
1430307,26416970,o+9gbX201ITS3MuMZYjuZUzV7sOoR57/0ttyX3eSWTA=
1430308,79633116,NZhNiXYhArpPquzrqh/kVj2IZ3QPRKm6FH90vsTlhiy=
1430309,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=


In [ ]:
devices.describe()

,userid,device
count,1430311,1430311
unique,481519,1363287
top,8158386,OqtiOn3PQUFp/QE5HMhvVww2jndmQ7ZTKlaRqnhdqBK=
freq,582,36


In [ ]:
devices[devices['device'].isna()]

,userid,device


In [ ]:
devices.groupby('userid').count()

,device
userid,
100000188,2
10000108,3
10000114,14
100001610,2
100001712,6
...,...
99995312,2
99997576,1
99998872,2


In [ ]:
devices[devices.duplicated(subset='userid', keep = False)].groupby('userid').count()

,device
userid,
100000188,2
10000108,3
10000114,14
100001610,2
100001712,6
...,...
99995186,2
99995312,2
99998872,2


In [ ]:
devices[devices.duplicated(subset='device', keep = False)].groupby('device').count()

,userid
device,
+++ivFTF+M/DxnA21MRSxuqZO/KUheIu0RXva/O41sq=,2
++53iBcrIz8AlMWkrmV+wM2Dd6sDURson4MMwahyFRW=,3
++GVp0u/WSB131/gJpU9ZLVswJDyz4q2O9iL14FSbp8=,2
++HhyYz77gILTvO2aogRTSpl0N6DdKWe7C5CK0IsL/E=,2
++IPuvUfesRBaGrhqCg/zho8sMyMBITh5vBaXgtudHy=,2
...,...
zzi/SHPqugydoH/NCERZcs1AdvmE4wkj15i2Oc016ku=,2
zzkrgJNWfevR2qHxLlvvHpP6zn79hz22jco9OENsp5y=,2
zzr+Gj7BPyvux9jp1XDLpxUY2qUz9APDuNwYdwKKwsq=,3


### Orders

In [ ]:
orders

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
...,...,...,...
620942,1956636054,14222102,70940136
620943,1956642760,2978154,60766142
620944,1956651496,221974772,15541938
620945,1956660812,152533590,80986700


In [ ]:
orders.groupby('buyer_userid').count()

,orderid,seller_userid
buyer_userid,,
100000188,1,1
10000108,1,1
100001610,1,1
100001712,5,5
100001882,1,1
...,...,...
99995186,1,1
99995312,1,1
99998872,1,1


### Quan hệ giữa các bảng

In [ ]:
temp = pd.merge(devices, bank_accounts,how='left', on=['userid'])

In [ ]:
temp = pd.merge(temp, credit_card, how='left', on=['userid'])
temp

,userid,device,bank_account,credit_card
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=,3242920139,393922xxxxxx4431|11-2021
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=,9380814192,NaN
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,0182401830,320808xxxxxx3923|07-2020
3,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,0182401830,322421xxxxxx9202|04-2020
4,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=,9120942033,320808xxxxxx3923|07-2020
...,...,...,...,...
2063978,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=,0313829392,NaN
2063979,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=,9900303042,NaN
2063980,19883370,dZ7YqVTprtTvLT4qS0YEWsn9EJahBJVQtTLKC6WaHZA=,228402024329822,NaN
2063981,698234,Mtmaool7T0CUyHJD112gAw6OiTl4tVSAPK4njG9B56y=,2322013980,NaN


In [ ]:
temp[temp['credit_card'].isna() & temp['bank_account'].isna()]

,userid,device,bank_account,credit_card
8,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=,NaN,NaN
10,62907954,Qy0OTEs+/dmv+aBN4Qw4mQScGsAAp435kHF69jdnQUm=,NaN,NaN
13,133160044,r7MzY5w6X7pz0kKqhAG4cIoU1CmcA/oG8BZ7CCBnK5E=,NaN,NaN
15,29345296,TGaB5cwOkWSi0YEvqhcZYyrUn1HAbi/Nj6gU3OmsCTO=,NaN,NaN
16,62003492,4bJgIFbzMZ3ywzrGdt5iGtkA3U8YxM/VmtEo9GMxxsm=,NaN,NaN
...,...,...,...,...
2063889,23145098,wu31Brqpy5RCJ1McbxrHrVletQv0+pUn7T0lzqotgzI=,NaN,NaN
2063897,25418616,PGMXs6WQ4mMnkird/SPvR0Eh2vwzpHY++3KWFReaPoK=,NaN,NaN
2063920,70441986,q/uqu52fGCn4u/KgKN/oGq2KPSC5XxGeFSAXtOYboa8=,NaN,NaN
2063921,11897402,OrvQuNxbnKDLhGyUspFxAx/YSlqNZhJdwZLCtz9ZvF4=,NaN,NaN


Người bán có một số không có tài khoản ngân hàng/

In [ ]:
temp2 = pd.merge(orders, bank_accounts, how='left', left_on='seller_userid', right_on='userid')
temp2 = pd.merge(temp2, credit_card, how='left', left_on='seller_userid', right_on='userid')
temp2 = pd.merge(temp2, devices, how='left', left_on='seller_userid', right_on='userid')
temp2

,orderid,buyer_userid,seller_userid,userid_x,bank_account,userid_y,credit_card,userid,device
0,1953278092,47388162,20822974,20822974,9409184444,NaN,NaN,20822974,VYTZXmDy0XH4A0PmT+LOTbHqDBZHIcAh2wqvim+EJ7c=
1,1953278092,47388162,20822974,20822974,9409184444,NaN,NaN,20822974,wtO13KzMDlo0NFVd8eyhSQDlNErtJe7mCsIYwwqMFcc=
2,1953278092,47388162,20822974,20822974,9409184444,NaN,NaN,20822974,H8vcR4qskt13dYaEHG7uI/DVOBFRHICGI0l5XwUT1+c=
3,1953278092,47388162,20822974,20822974,9409184444,NaN,NaN,20822974,loWi7gSa4JftesNF79hx/el2/mdeGk1zin/+fdRR4fW=
4,1953278092,47388162,20822974,20822974,9409184444,NaN,NaN,20822974,qeeY/PwvWCLyPpbomcRJscxjkyHSZzyOm79kNtCc0IS=
...,...,...,...,...,...,...,...,...,...
17278860,1956679514,54374608,10385548,10385548,9290009322,NaN,NaN,10385548,43GeMpRslrC38r64Ipf4VhLxgRkXIo16GFhQcV0ISLm=
17278861,1956679514,54374608,10385548,10385548,9290009322,NaN,NaN,10385548,OI+u5GmjBo194O4m/dqBsnH96XHmdoZuQhN/ORyd1m8=
17278862,1956679514,54374608,10385548,10385548,9290009322,NaN,NaN,10385548,7dPsed046Iiz8Pukml6Y4mhfEDY+6JmvUQaM/ssNQqK=
17278863,1956679514,54374608,10385548,10385548,9290009322,NaN,NaN,10385548,Nk86ZwTtM/jl+QeB60KcIi1rEwUDzBccqqaJbUUQHsm=


In [ ]:
temp2[temp2['device'].isna()]

,orderid,buyer_userid,seller_userid,userid_x,bank_account,userid_y,credit_card,userid,device
2522,1954180082,21712370,153974628,153974628,0934222000,NaN,NaN,NaN,NaN
2889,1954478166,141797242,76558244,NaN,NaN,NaN,NaN,NaN,NaN
2938,1954529620,2218588,183582984,NaN,NaN,NaN,NaN,NaN,NaN
2939,1954545574,205591650,117091086,NaN,NaN,NaN,NaN,NaN,NaN
3887,1955162986,121922588,6108522,6108522,9322204348,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
17272887,1954070008,170419480,164170624,NaN,NaN,NaN,NaN,NaN,NaN
17276113,1955278962,228783846,128256442,NaN,NaN,NaN,NaN,NaN,NaN
17277447,1956134582,53492158,137265350,NaN,NaN,NaN,NaN,NaN,NaN
17277551,1956258002,10358262,133611704,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
temp3[temp3['device'].isnull()]

,orderid,buyer_userid,seller_userid,userid,device
1533,1953533862,157781268,153156894,NaN,NaN
3050,1955649840,233795460,17829614,NaN,NaN
3251,1956599292,235658398,122593416,NaN,NaN
4883,1956505936,75421990,19928070,NaN,NaN
5265,1954366120,235444002,165182356,NaN,NaN
...,...,...,...,...,...
1705036,1954497806,57857506,80933100,NaN,NaN
1705272,1955832124,182423394,43330010,NaN,NaN
1705654,1954043698,234136914,52325894,NaN,NaN
1705843,1954681166,143717580,20500834,NaN,NaN


In [ ]:
temp4 = pd.merge(orders, devices, left_on= 'seller_userid', right_on = 'userid' ,how='left')
temp4

,orderid,buyer_userid,seller_userid,userid,device
0,1953278092,47388162,20822974,20822974,VYTZXmDy0XH4A0PmT+LOTbHqDBZHIcAh2wqvim+EJ7c=
1,1953278092,47388162,20822974,20822974,wtO13KzMDlo0NFVd8eyhSQDlNErtJe7mCsIYwwqMFcc=
2,1953278092,47388162,20822974,20822974,H8vcR4qskt13dYaEHG7uI/DVOBFRHICGI0l5XwUT1+c=
3,1953278092,47388162,20822974,20822974,loWi7gSa4JftesNF79hx/el2/mdeGk1zin/+fdRR4fW=
4,1953278092,47388162,20822974,20822974,qeeY/PwvWCLyPpbomcRJscxjkyHSZzyOm79kNtCc0IS=
...,...,...,...,...,...
5482861,1956679514,54374608,10385548,10385548,43GeMpRslrC38r64Ipf4VhLxgRkXIo16GFhQcV0ISLm=
5482862,1956679514,54374608,10385548,10385548,OI+u5GmjBo194O4m/dqBsnH96XHmdoZuQhN/ORyd1m8=
5482863,1956679514,54374608,10385548,10385548,7dPsed046Iiz8Pukml6Y4mhfEDY+6JmvUQaM/ssNQqK=
5482864,1956679514,54374608,10385548,10385548,Nk86ZwTtM/jl+QeB60KcIi1rEwUDzBccqqaJbUUQHsm=


In [ ]:
temp4[temp4['userid'].isnull()]

,orderid,buyer_userid,seller_userid,userid,device
768,1954180082,21712370,153974628,NaN,NaN
963,1954478166,141797242,76558244,NaN,NaN
976,1954529620,2218588,183582984,NaN,NaN
977,1954545574,205591650,117091086,NaN,NaN
1414,1955162986,121922588,6108522,NaN,NaN
...,...,...,...,...,...
5481083,1954070008,170419480,164170624,NaN,NaN
5481933,1955278962,228783846,128256442,NaN,NaN
5482537,1956134582,53492158,137265350,NaN,NaN
5482582,1956258002,10358262,133611704,NaN,NaN


In [ ]:
temp5 = pd.merge(orders, bank_accounts, left_on = 'seller_userid', right_on = 'userid', how = 'left')
temp5

,orderid,buyer_userid,seller_userid,userid,bank_account
0,1953278092,47388162,20822974,20822974,9409184444
1,1953278092,47388162,20822974,20822974,8380334004
2,1953278092,47388162,20822974,20822974,2082389004
3,1953295120,26855196,16416890,16416890,2033180891
4,1953295120,26855196,16416890,16416890,239002092381820
...,...,...,...,...,...
1438773,1956642760,2978154,60766142,60766142,3100299282
1438774,1956651496,221974772,15541938,15541938,9828232992
1438775,1956651496,221974772,15541938,15541938,9822920920
1438776,1956660812,152533590,80986700,80986700,9290908900


In [ ]:
temp5[temp5['bank_account'].isnull()]

,orderid,buyer_userid,seller_userid,userid,bank_account
18,1953480060,130236220,77145726,NaN,NaN
74,1953756734,56942532,37178008,NaN,NaN
133,1954025318,47488278,155771772,NaN,NaN
183,1954291258,116866314,76950972,NaN,NaN
201,1954460052,127068920,90746286,NaN,NaN
...,...,...,...,...,...
1438698,1956258002,10358262,133611704,NaN,NaN
1438720,1956415556,12361456,193529670,NaN,NaN
1438737,1956499166,81742130,26970898,NaN,NaN
1438759,1956561772,158271248,128755420,NaN,NaN


Két quả trên cho thấy vẫn tồn tại các seller không có tài khoản ngân hàng.

In [ ]:
temp6 = pd.merge(orders, credit_card, left_on = 'seller_userid', right_on = 'userid', how = 'left')
temp6

,orderid,buyer_userid,seller_userid,userid,credit_card
0,1953278092,47388162,20822974,NaN,NaN
1,1953295120,26855196,16416890,NaN,NaN
2,1953306402,121296714,28477978,NaN,NaN
3,1953314712,131221930,72837912,NaN,NaN
4,1953381964,183398314,28423332,NaN,NaN
...,...,...,...,...,...
671735,1956636054,14222102,70940136,NaN,NaN
671736,1956642760,2978154,60766142,NaN,NaN
671737,1956651496,221974772,15541938,NaN,NaN
671738,1956660812,152533590,80986700,80986700,324383xxxxxx9890|04-2019


In [ ]:
temp6.isnull().sum()

orderid               0
buyer_userid          0
seller_userid         0
userid           561894
credit_card      561894
dtype: int64

In [ ]:
temp7 = orders[orders['buyer_userid'] == orders['seller_userid']]
temp7

,orderid,buyer_userid,seller_userid
